In [72]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
import matplotlib.pyplot as plt
import math
import csv
import random

# 导入数据
訓練集與測試集的處理方法跟 logistic regression 一模一樣，然而因為 generative model 有可解析的最佳解，因此不必使用到 development set

In [73]:
#文件位置
X_train_fpath = 'hw2_data/X_train.csv'
Y_train_fpath = 'hw2_data/Y_train.csv'
X_test_fpath = 'hw2_data/X_test.csv'
output_fpath = 'hw2_data/预测结果_{}.csv'

#读文件 + 定好数据类型为float
data = pd.read_csv(X_train_fpath)
X_train =data.iloc[:, 1:].to_numpy(dtype = np.float)
data = pd.read_csv(Y_train_fpath)
Y_train =data.iloc[:, 1:].to_numpy(dtype = np.float)
data = pd.read_csv(X_test_fpath)
X_test =data.iloc[:, 1:].to_numpy(dtype = np.float)

# 均值 和 方差
在 generative model 中，我們需要分別計算兩個類別內的資料平均與共變異

两个随机变量之间的协方差公式：
$$cov(X, Y) = E[(X-EX)(Y-EY)] = {{1 \over n}\sum_{i=1}^n{(x_i-\mu_x)(y_i-\mu_y)}}$$

m组随机变量，每组N个样本，每组之间求协方差，可以建立一个 m X m 阶协方差矩阵C：
$$ C = {{1 \over N}(X-\mu_X)^T(X-\mu_X)}$$
C中每项的计算公式：
$$C_{(i,j)}={1 \over N}(X_i - \mu_i)(X_j - \mu_j)$$ 
其中N为样本数量

In [74]:
data_dim = X_train.shape[1] #数据集的维度

# 将X_train按照Y_train的数值分成2组，每组都是510个维度，但每组包含的样本数不同
X_train_0 = np.array([x for x, y in zip(X_train, Y_train) if y == 0])
X_train_1 = np.array([x for x, y in zip(X_train, Y_train) if y == 1])

# 每组按列算均值
mean_0 = np.mean(X_train_0, axis = 0).reshape(1,-1)
mean_1 = np.mean(X_train_1, axis = 0).reshape(1,-1)

#答案的计算方法, 没用矩阵算协方差矩阵，下面修正，加速
'''
# 每组的协方差矩阵初始化
cov_0 = np.zeros((data_dim, data_dim))
cov_1 = np.zeros((data_dim, data_dim))

# 计算每组的协方差矩阵
for x in X_train_0:
    cov_0 += np.dot(np.transpose([x - mean_0]), [x - mean_0]) / X_train_0.shape[0]
for x in X_train_1:
    cov_1 += np.dot(np.transpose([x - mean_1]), [x - mean_1]) / X_train_1.shape[0]
'''
# 计算每组的协方差矩阵
cov_0 =  np.dot(np.transpose(X_train_0 - mean_0), X_train_0 - mean_0) / X_train_0.shape[0]
cov_1 =  np.dot(np.transpose(X_train_1 - mean_1), X_train_1 - mean_1) / X_train_1.shape[0]

# 假设两组的协方差矩阵一样，将两组协方差矩阵加权平均作为共同的协方差矩阵
cov = (cov_0 * X_train_0.shape[0] + cov_1 * X_train_1.shape[0]) / (X_train_0.shape[0] + X_train_1.shape[0])

# 权重 和 bias

權重矩陣與偏差向量可以直接被計算出來，算法可以參考李宏毅老師教學投影片第 33 頁

cov是对称矩阵，所以一定可以对角化，且存在正交向量Q, 得
$$ \Lambda = P^{-1} cov P = Q^{-1} cov Q = Q^T cov Q $$
得
$$cov = (Q^T)^{-1} \Lambda Q^{-1} = Q \Lambda Q^T = USV$$

所以 $$ cov^{-1} = (USV)^{-1} = V^{-1} S^{-1} U^{-1} = V^T S^{-1} U^T = inv$$

In [75]:
# 计算协方差矩阵的逆矩阵，由于协方差矩阵可能是不可逆的，不能直接求，需要通过奇异值分解算
u, s, v = np.linalg.svd(cov, full_matrices=False) #奇异值分解, 算特征值和特征向量
inv = np.matmul(v.T * 1 / s, u.T) #得到cov的逆矩阵

## 直接算w 和 b
$$ probability = \sigma(z) = {1 \over {1+e^{-z}}} $$    
其中：   $$ z = f_{w,b}(x)= \sum_i X_iw_i + b $$
其中： 
$$ w= (\mu_0 - \mu_1)inv $$

$$ b = -{1\over2}\mu_0 inv \mu_0^T + {1\over2}\mu_1 inv \mu_1^T + ln{N_0\over N_1}$$

In [76]:
eps = 1e-8 #一个非常小的数, 防止分母为0
def _sigmoid(z):#算概率
    return np.clip(1 / (1.0 + np.exp(-z)), eps, 1 - eps) 
    #将数组中的元素限制在eps, 1-eps之间，大于1-eps的就使得它等于1-eps，小于eps,的就使得它等于eps

def _f(X, w, b):# 每一行的可能性
    ans = _sigmoid(np.matmul(X, w) + b)
    ans = ans.reshape(-1, 1) #形状变成列向量
    return ans

def _predict(X, w, b): #将结果四舍五入, <0.5为0, >0.5为1
    return np.round(_f(X, w, b)).astype(np.int)

def _accuracy(Y_pred, Y_label):# 算是否准确
    acc = 1 - np.mean(np.abs(Y_pred - Y_label))
    return acc

In [77]:
#答案的矩阵形状不对
w = np.dot(mean_0 - mean_1, inv).reshape(-1,1)
b =  (-0.5) * np.dot(mean_0, np.dot(inv, mean_0.T)) + 0.5 * np.dot(mean_1, np.dot(inv, mean_1.T))\
    + np.log(float(X_train_0.shape[0]) / X_train_1.shape[0]) 

# Compute accuracy on training set
Y_train_pred = 1 - _predict(X_train, w, b)
print('训练准确率: {}'.format(_accuracy(Y_train_pred, Y_train)))

训练准确率: 0.8754607785314067


# 预测

預測測試集的資料標籤

In [78]:
predictions = 1 - _predict(X_test, w, b)
with open(output_fpath.format('生成模型'), 'w') as f:
    f.write('id,label\n')
    for i, label in  enumerate(predictions):
        f.write('{},{}\n'.format(i, label[0]))

#列出最有影响力的因子
w1 = w.reshape(w.shape[0])
ind = np.argsort(np.abs(w1))[::-1]
with open(X_test_fpath) as f:
    content = f.readline().strip('\n').split(',')
features = np.array(content)
for i in ind[0:10]:
    print(features[i], w[i][0])

 Not in universe -3.6705766095510626
 5th or 6th grade -3.2322470678627724
21 -3.0797095324285237
 Self-employed-not incorporated -2.7075818398216143
12 -2.5280813629981367
 Bachelors degree(BA AB BS) -2.460140471401246
36 -2.2042641642179888
 Some college but no degree -2.083672045959247
 Social services 2.024659527334734
 Panama -1.9747868643607944
